### 🛠️ Initialize Notebook Variables

**Only modify entries under _USER CONFIGURATION_.**

**Run steps 1 & 2 manually, make the necessary changes for step 3, then continue with step 4.**

This OAuth 3rd party APIM sample demonstrates integration with Spotify API using OAuth 2.0 authentication and JWT validation.

**Prerequisites required:**
- Spotify Developer account and registered application
- Environment variables: `SPOTIFY_CLIENT_ID` and `SPOTIFY_CLIENT_SECRET`
- Manual OAuth connection setup (detailed in deployment steps)

In [ ]:
import utils
from apimtypes import *
import os

# ------------------------------
#    USER CONFIGURATION
# ------------------------------
rg_location = 'eastus2'
index       = 1
deployment  = INFRASTRUCTURE.SIMPLE_APIM
api_prefix  = 'oauth-'  # Prefix for API names                                      # ENTER A PREFIX FOR THE APIS TO REDUCE COLLISION POTENTIAL WITH OTHER SAMPLES
tags        = ['oauth-3rd-party', 'jwt', 'credential-manager', 'policy-fragment']   # ENTER DESCRIPTIVE TAG(S)



# ------------------------------
#    SYSTEM CONFIGURATION
# ------------------------------

# Create the notebook helper with JWT support
sample_folder    = 'oauth-3rd-party'
rg_name          = utils.get_infra_rg_name(deployment, index)
supported_infras = [INFRASTRUCTURE.AFD_APIM_PE, INFRASTRUCTURE.APIM_ACA, INFRASTRUCTURE.SIMPLE_APIM]
nb_helper        = utils.NotebookHelper(sample_folder, rg_name, rg_location, deployment, supported_infras, True, index = index)

# OAuth credentials (required environment variables)
client_id        = os.getenv('SPOTIFY_CLIENT_ID')
client_secret    = os.getenv('SPOTIFY_CLIENT_SECRET')

# Validate OAuth credentials
if not client_id or not client_secret:
    utils.print_error('Please set the SPOTIFY_CLIENT_ID and SPOTIFY_CLIENT_SECRET environment variables in the root .env file before running this notebook.')
    raise ValueError('Missing Spotify OAuth credentials')

# Define the APIs and their operations and policies

# Set up the named values
nvs: List[NamedValue] = [
    NamedValue(nb_helper.jwt_key_name, nb_helper.jwt_key_value_bytes_b64, True),
    NamedValue('MarketingMemberRoleId', Role.MARKETING_MEMBER)
]

# Load policy definitions
pol_artist_get_xml  = utils.read_policy_xml('artist_get.xml', sample_name=sample_folder)
pol_spotify_api_xml = utils.read_and_modify_policy_xml('spotify_api.xml', {
    'jwt_signing_key': '{{' + nb_helper.jwt_key_name + '}}', 
    'marketing_member_role_id': '{{MarketingMemberRoleId}}'
}, sample_folder)

# Define template parameters for artist ID
blob_template_parameters = [
    {
        "name": "id",
        "description": "The Spotify ID of the artist",
        "type": "string",
        "required": True
    }
]

# Define API operations

# API 1: Spotify
spotify_path       = f'{api_prefix}spotify'
spotify_artist_get = GET_APIOperation2('artists-get', 'Artists', '/artists/{id}', 'Gets the artist by their ID', pol_artist_get_xml, templateParameters = blob_template_parameters)
spotify            = API(spotify_path, 'Spotify', spotify_path, 'This is the API for interactions with the Spotify REST API', pol_spotify_api_xml, [spotify_artist_get], tags)

# APIs Array
apis: List[API] = [spotify]

utils.print_ok('Notebook initialized')

### 🚀 Deploy Infrastructure and APIs

Creates the bicep deployment into the previously-specified resource group. A bicep parameters, `params.json`, file will be created prior to execution.

In [ ]:
# Build the bicep parameters
bicep_parameters = {
    'apis'         : {'value': [api.to_dict() for api in apis]},
    'namedValues'  : {'value': [nv.to_dict() for nv in nvs]},
    'clientId'     : {'value': client_id},
    'clientSecret' : {'value': client_secret}
}

# Deploy the sample
output = nb_helper.deploy_sample(bicep_parameters)

if output.success:
    # Extract deployment outputs for testing
    apim_name                  = output.get('apimServiceName', 'APIM Service Name')
    apim_gateway_url           = output.get('apimResourceGatewayURL', 'APIM API Gateway URL')
    apim_apis                  = output.getJson('apiOutputs', 'APIs')
    spotify_oauth_redirect_url = output.get('spotifyOAuthRedirectUrl', 'OAuth Redirect URL')

    utils.print_ok('Deployment completed successfully')
else:
    utils.print_error("Deployment failed!")
    raise SystemExit(1)

### Configure OAuth Connection (Manual Steps)

**The following steps are manual and required before testing:**

### 3.1 Update Spotify Redirect URL
1. Open [Spotify Developer Dashboard](https://developer.spotify.com/dashboard)
2. Click on your APIM application
3. Press **Edit** and remove the temporary localhost Redirect URI
4. Add the `OAuth Redirect URL` from the deployment output above
5. Press **Save**

### 3.2 Authenticate APIM with Spotify
1. Open [Azure Portal](https://portal.azure.com) and navigate to your API Management instance
2. Go to **APIs** > **Credential manager** > **spotify**
3. Click **Connections** > **spotify-auth** (should show "Error" status)
4. Click ellipsis (...) > **Login** and authorize the connection
5. Press **Refresh** to verify "Connected" status

### ✅ Verify API Request Success

Assert that the deployment was successful by making simple calls to APIM. 

❗️ If the infrastructure shields APIM and requires a different ingress (e.g. Azure Front Door), the request to the APIM gateway URl will fail by design. Obtain the Front Door endpoint hostname and try that instead.

In [ ]:
# Test and verify OAuth integration with Spotify
import json
from apimrequests import ApimRequests
from apimtesting import ApimTesting
from users import UserHelper
from authfactory import AuthFactory

# Initialize testing framework
tests = ApimTesting("OAuth 3rd Party (Spotify) Sample Tests", sample_folder, deployment)
api_subscription_key = apim_apis[0]['subscriptionPrimaryKey']

# Create JWT token for Marketing Member role
encoded_jwt_token_marketing_member = AuthFactory.create_symmetric_jwt_token_for_user(
    UserHelper.get_user_by_role(Role.MARKETING_MEMBER), 
    nb_helper.jwt_key_value
)
utils.print_info(f'JWT token for Marketing Member:\n{encoded_jwt_token_marketing_member}')

# Get the appropriate endpoint URL for testing
endpoint_url = utils.test_url_preflight_check(deployment, rg_name, apim_gateway_url)

# Test Spotify API integration
reqs = ApimRequests(endpoint_url, api_subscription_key)
reqs.headers['Authorization'] = f'Bearer {encoded_jwt_token_marketing_member}'

# Test artist lookup (Taylor Swift's Spotify Artist ID)
artist_id = '06HL4z0CvFAxyc27GXpf02'
output = reqs.singleGet(f'{spotify_path}/artists/{artist_id}', msg = 'Calling the Spotify Artist API via API Management Gateway URL.')

artist = json.loads(output)
tests.verify(artist['name'], 'Taylor Swift')
utils.print_info(f'{artist["name"]} has a popularity rating of {artist["popularity"]} with {artist["followers"]["total"]:,} followers on Spotify.')

# Test unauthorized access (should fail with 401)
reqsNoApiSubscription = ApimRequests(endpoint_url)
output = reqsNoApiSubscription.singleGet(f'{spotify_path}/artists/{artist_id}', msg = 'Calling the Spotify Artist API without API subscription key. Expect 401.')
outputJson = utils.get_json(output)
tests.verify(outputJson['statusCode'], 401)
tests.verify(outputJson['message'], 'Access denied due to missing subscription key. Make sure to include subscription key when making requests to an API.')

tests.print_summary()
utils.print_ok('✅ All OAuth integration tests completed successfully!')